# Regresion logistica

## Importamos las bibiliotecas que utilizaremos a lo largo del notebook

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from preprocessing import prepararSetDeDatos
from preprocessing import svmPreprocessing

## Importamos los datos y los procesamos

In [ ]:
X = pd.read_csv('Datasets/tp-2020-2c-train-cols2.csv')
y = pd.read_csv('Datasets/tp-2020-2c-train-cols1.csv')
X,y = prepararSetDeDatos(X,y)

In [ ]:
X_rl = svmPreprocessing(X) #crear prepocessing

## Funciones Auxiliares

Creamos una funcion que obtiene mediante grid search y K-Fold cross validation el tipo de regularizacion que maximiza la metrica de AUC ROC para el modelo de SVM.

In [ ]:
def obtenerMejoresParametros(datosPreprocesados):
    mejor_valor = 0
    mejor_regularizacion = None
    y_array=np.array(y)
    for regularizacion in ["l2","none", "l1", "elasticnet"]:
            if(regularizacion == "elasticnet"):
                l1_ratio = 0.5
            else:
                l1_ratio = None
            kf = StratifiedKFold(n_splits=5)
            metricas = []
            for fold_idx, (train_index, test_index) in enumerate(kf.split(datosPreprocesados, y_array)):
                rl = LogisticRegression(penalty = regularizacion, max_iter = 5000, solver = "saga", l1_ratio = l1_ratio)
                rl.fit(datosPreprocesados[train_index], y_array[train_index].ravel())
                predicciones = rl.predict(datosPreprocesados[test_index])
                score_obtenida = roc_auc_score(y_array[test_index],predicciones)
                metricas.append(score_obtenida)

            if np.mean(metricas) >= mejor_valor:
                mejor_valor = np.mean(metricas)
                mejor_regularizacion = regularizacion
            
    return mejor_valor, mejor_regularizacion

### Dividimos el set de datos en sets de training y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_rl, y, test_size=0.25, random_state=0)

### Buscamos los mejores parametros

In [ ]:
mejor_valor, mejor_regularizacion = obtenerMejoresParametros(X_rl)

In [ ]:
print(f"El mejor valor fue de AUC fue: {mejor_valor}")
print(f"La regularizacion encontrada que maximiza el AUC fue: {mejor_regularizacion}")

### Evaluamos las metricas

In [ ]:
rl = LogisticRegression(penalty = mejor_regularizacion, max_iter = 1000)
rl.fit(X_train, y_train)
y_pred = rl.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['No vuelve','Vuelve']))

### Matriz de confusion

In [ ]:
fig, ax = plt.subplots(dpi =150)   
sns.heatmap(confusion_matrix(y_test, y_pred), annot = True, vmin = 0, yticklabels=["Volveria","No volveria"], xticklabels=["Volveria", "No Volveria"], ax=ax)
ax.set_title("Matriz de confusión de Regresion Logistica")
ax.set_xlabel("Predicho")
ax.set_ylabel("Real")

### Graficamos la curva ROC

In [ ]:
fpr_rl_test,tpr_rl_test,thresholds_rl_test = roc_curve(y_test,rl.predict_proba(X_test)[:,1])

zero_test = np.argmin(np.abs(thresholds_rl_test))

plt.plot(fpr_rl_test,tpr_rl_test,label="ROC Curve RL Test")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.plot(fpr_rl_test[zero_test],tpr_rl_test[zero_test],'o',markersize=10,label="threshold zero test",fillstyle="none",c="k",mew=2)

plt.legend(loc=4)
plt.show()

In [ ]:
auc_rl = roc_auc_score(y_test,rl.predict_proba(X_test)[:, 1])
print("AUC para Regresion Logistica: {:.3f}".format(auc_rl))